In [ ]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import numpy as np
import matplotlib as plt
import cv2
import math
import os
import keras
from keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Flatten, Reshape, Activation, Dense, UpSampling2D, LeakyReLU, Input
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import backend as K

In [ ]:
images=list()
for i in range(1,1000):
    a=cv2.imread("img_align_celeba/%06d.jpg"%i,1)
    a=cv2.resize(a,(a.shape[0]//2-1,a.shape[1]//2-1))
    a=a/255.0
    a=a.tolist()
    images.append(a)
images=np.asarray(images)

In [ ]:
def gen(inputs, image_shape):
    
    x=Dense((image_shape[0]//4) * (image_shape[1]//4) * 128)(inputs)
    x=Reshape((image_shape[0]//4, image_shape[1]//4, 128))(x)
    
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2DTranspose(filters = 128, kernel_size=5, strides=2, padding='same', activation = 'sigmoid')(x)
    
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2DTranspose(filters = 64, kernel_size=5, strides=2, padding='same', activation = 'sigmoid')(x)
    
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2DTranspose(filters = 32, kernel_size=5, strides=1, padding='same', activation = 'sigmoid')(x)
    
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2DTranspose(filters = 3, kernel_size=5, strides=1, padding='same', activation = 'sigmoid')(x)
        
    generator=Model(inputs, x)
    return generator

In [ ]:
def dis(inputs):
    x=inputs
    
    x=LeakyReLU(alpha=0.2)(x)
    x= Conv2D(filters= 32, kernel_size=5, strides=2, padding='same')(x)
    
    x=LeakyReLU(alpha=0.2)(x)
    x= Conv2D(filters= 32, kernel_size=5, strides=2, padding='same')(x)
    
    x=LeakyReLU(alpha=0.2)(x)
    x= Conv2D(filters= 32, kernel_size=5, strides=2, padding='same')(x)
    
    x=LeakyReLU(alpha=0.2)(x)
    x= Conv2D(filters= 32, kernel_size=5, strides=1, padding='same')(x)
    
    x=Flatten()(x)
    x=Dense(1, activation = 'sigmoid')(x)
    
    discriminator = Model(inputs, x)
    return discriminator

In [ ]:
def models(latent_size):
    input_shape=(images.shape[1],images.shape[2],images.shape[3])
    
    inputs = Input(shape=input_shape)
    discriminator=dis(inputs)
    discriminator.compile(loss = 'binary_crossentropy', optimizer=RMSprop(lr= 2e-4, decay=6e-8), metrics = ['accuracy'])
    
    inputs = Input(shape=(latent_size, ))
    generator=gen(inputs, input_shape)
    
    discriminator.trainable = False
    
    adversarial=Model(inputs, discriminator(generator(inputs)))
    adversarial.compile(loss='binary_crossentropy', optimizer = RMSprop(lr=2e-4*0.5,decay=6e-8*0.5))
    
    models=(generator, discriminator, adversarial)
    return models

In [ ]:
def train(models, epochs, batch_size, latent_size, save_interval=0):
    generator, discriminator, adversarial = models
    
    for i in range(epochs):
        rand_indices = np.random.randint(0, images.shape[0], size = batch_size)
        real = images[rand_indices]
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        fake = generator.predict(noise)
        
        x = np.concatenate((real,fake))
        
        y = np.ones(batch_size*2)
        y[batch_size:]=0.0
        y=np.reshape(y,(batch_size*2,1))
        
        history = discriminator.train_on_batch(x,y)
        
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        y=np.ones(batch_size)
        y=np.reshape(y,(batch_size,1))
        
        """aloss, aacc =""" 
        history1 = adversarial.train_on_batch(noise, y)
        
        print("Epoch : %d, [adversarial loss: %f] [discriminator loss: %f, acc: %f]"%(i+1, history1, history[0], history[1]))
        
        if save_interval!=0:
            if i%save_interval==0:
                noise = np.random.uniform(0, 1.0, size = [1, latent_size])
                n=generator.predict(noise)
                cv2.imwrite("%d.jpeg"%i,n[0])
                print("Saved!")

In [ ]:
model=models(100)
train(model, 1000, 64, 100, 10)